# Data prepation


Imports:

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


Load all data (set path first)

In [2]:
original_data_path = "C:/Users/peter/Documents/Peter school/Leerjaar 3/VU/Machine Learning/data/"
or_movies = pd.read_csv(original_data_path+'IMDb movies.csv', delimiter=",", low_memory=False)
or_names = pd.read_csv(original_data_path+'IMDb names.csv', delimiter=",", low_memory=False)
or_ratings = pd.read_csv(original_data_path+'IMDb ratings.csv', delimiter=",", low_memory=False)
or_title_principles = pd.read_csv(original_data_path+'IMDb title_principals.csv', delimiter=",", low_memory=False)

## remove the columns with more than 50% NA-values (stored in new df)

In [3]:
#make function to count NA-values
na_counts_movies = or_movies.isna().sum() / len(or_movies) * 100
na_counts_names = or_names.isna().sum() / len(or_names) * 100
na_counts_ratings = or_ratings.isna().sum() / len(or_ratings) * 100
na_counts_title_principles = or_title_principles.isna().sum() / len(or_title_principles) * 100

#get all indexes
movie_columns_to_drop = na_counts_movies[na_counts_movies > 50].index
name_columns_to_drop = na_counts_names[na_counts_names > 50].index
ratings_columns_to_drop = na_counts_ratings[na_counts_ratings > 50].index
title_principles_columns_to_drop = na_counts_title_principles[na_counts_title_principles > 50].index

#Make new df and drop the column index
adj_movies = or_movies.drop(columns=movie_columns_to_drop)
adj_name_columns = or_movies.drop(columns=movie_columns_to_drop)
adj_ratings = or_movies.drop(columns=movie_columns_to_drop)
adj_title_principles = or_movies.drop(columns=movie_columns_to_drop)


# remove columns we dont want to use

In [4]:
drop = ["imdb_title_id", "original_title", "date_published", "description", "votes", "reviews_from_users",
        "reviews_from_critics", "production_company"]
adj_movies = adj_movies.drop(columns = drop)
adj_movies.head()

,title,year,genre,duration,country,language,director,writer,production_company,actors,avg_vote
0,Miss Jerry,1894,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",5.9
1,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",6.1
2,Den sorte drøm,1911,Drama,53,"Germany, Denmark",NaN,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",5.8
3,Cleopatra,1912,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",5.2
4,L'Inferno,1911,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",7.0


# new feature title length and remove title

In [5]:
adj_movies.loc[:,'title length'] = adj_movies.loc[:,'title'].str.replace(' ', '').str.len()
adj_movies = adj_movies.drop(columns = ["title"])
adj_movies.head()

,year,genre,duration,country,language,director,writer,production_company,actors,avg_vote,title length
0,1894,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",5.9,9
1,1906,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",6.1,22
2,1911,Drama,53,"Germany, Denmark",NaN,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",5.8,12
3,1912,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",5.2,9
4,1911,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",7.0,9


# Derive new features from existing ones

## Country  (werkt nog niet)

In [8]:
#takes all country and put in array
countries = set(",".join(adj_movies["country"].str.replace(", ", ",").to_list()).split(","))
new_columns = ["country_"+country for country in countries]
new_columns.sort()
new_columns

TypeError: sequence item 53010: expected str instance, float found

## genres (apply on-hot encoding) (creates 25 new features)

In [7]:
#extract all genres
genres = set(",".join(adj_movies["genre"].str.replace(", ", ",").to_list()).split(","))
new_columns = ["genre_"+genre for genre in genres]
print(len(new_columns))

#apply one-hot encoding for genres
for column_name in new_columns:
    genre = column_name.replace("genre_", "")
    adj_movies[column_name] = adj_movies["genre"].str.contains(genre).astype(int)
del adj_movies["genre"]
adj_movies.head()



25


,year,duration,country,language,director,writer,production_company,actors,avg_vote,title length,...,genre_Sci-Fi,genre_Drama,genre_Romance,genre_Documentary,genre_War,genre_Reality-TV,genre_Thriller,genre_History,genre_Film-Noir,genre_Family
0,1894,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",5.9,9,...,0,0,1,0,0,0,0,0,0,0
1,1906,70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",6.1,22,...,0,1,0,0,0,0,0,0,0,0
2,1911,53,"Germany, Denmark",NaN,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",5.8,12,...,0,1,0,0,0,0,0,0,0,0
3,1912,100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",5.2,9,...,0,1,0,0,0,0,0,1,0,0
4,1911,68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",7.0,9,...,0,1,0,0,0,0,0,0,0,0


## language *hier moet nog wat worden gefixt kon het niet bedenken zo snel*

In [11]:
# remove NaN
languages = adj_movies[~adj_movies["language"].isnull()]["language"]

# remove None
languages = languages[languages!="None"]

#extract all languages in array
languages = list(set(",".join(languages.str.replace(", ", ",").to_list()).split(",")))
languages

#make list of uncommomen languages (less than 10 movies)

for column_name in languages:
    if adj_movies["language_"+column_name].sum() < 10:
        #add to uncommen array

#in adj movie replace all uncommen languages with language_other ofzo iets
# vergeet niet language_other toe te voegen aan languages zodat het in onehot coding werkt

#apply one-hot-encoding
for column_name in languages:
    genre = column_name.replace("language_", "")
    adj_movies[column_name] = adj_movies["language"].str.contains(language).astype(int)
del adj_movies["language"]
adj_movies.head()


KeyError: 'language_Shanghainese'

## Actor we kunnen hiervoor de top 250 doen en dan tellen of top 100 one-hot coding

In [ ]:
#code

## Writers kunnen hiervoor top  x writer doen en dan tellen of top one-hot coding

In [ ]:
#code

## Director zelfde verhaal

## split data
#### split op 80/20 dit zorgt voor ong 12.5k test set

In [ ]:
taget_column = adj_movies["avg_vote"]
features = adj_movies.loc[:, adj_movies.columns != 'avg_vote']

x_train, x_test, y_train, y_test = train_test_split(features, taget_column, test_size=0.2)

# exporteer weet niet of het klopt maar nu uitgezet

In [ ]:
#export
#x_train.to_csv("x_train")
#x_test.to_csv("x_test")

#y_train.to_csv("y_train")
#y_test.to_csv("y_test")